# Exploratory Data Analysis with SQL

The purpose here is to use some queries to explore the data.


In [2]:
!pip install seaborn

In [3]:
# Sqlite is a library that implements a SQL database engine. 
import sqlite3
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#connection
conn=sqlite3.connect('database.sqlite')
cursor = conn.cursor()
print("Database connection established successfully.")

Database connection established successfully.


In [6]:
#sqlite_master is a table with database schema
schema=pd.read_sql(""" SELECT *
                FROM sqlite_master
                WHERE type='table';""",
           conn)

schema

,type,name,tbl_name,rootpage,sql
0,table,Country,Country,2,"CREATE TABLE Country (\n CountryCode TEXT,\..."
1,table,CountryNotes,CountryNotes,186,CREATE TABLE CountryNotes (\n Countrycode T...
2,table,Series,Series,948,"CREATE TABLE Series (\n SeriesCode TEXT,\n ..."
3,table,Indicators,Indicators,4448,CREATE TABLE Indicators (\n CountryName TEX...
4,table,SeriesNotes,SeriesNotes,1317550,CREATE TABLE SeriesNotes (\n Seriescode TEX...
5,table,Footnotes,Footnotes,1317587,CREATE TABLE Footnotes (\n Countrycode TEXT...


## <span style='color:DarkGoldenrod'> Let's explore the Indicators table.  </span>

# 1- Selecting

In [12]:
# check the head
indic=pd.read_sql("""SELECT *
               FROM Indicators;""",
           conn)

indic.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [13]:
# how many rows?
indic.shape # 5656458 rows, 6 columns

(5656458, 6)

In [ ]:
# checking for missing values in one column ('IndicatorName')
print(indic['IndicatorName'].isnull().sum())

# check datatypes (for all columns)
indic.info()

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5656458 entries, 0 to 5656457
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CountryName    object 
 1   CountryCode    object 
 2   IndicatorName  object 
 3   IndicatorCode  object 
 4   Year           int64  
 5   Value          float64
dtypes: float64(1), int64(1), object(4)
memory usage: 258.9+ MB


In [15]:
# if I need filter not null
filter=pd.read_sql("""SELECT *
               FROM Indicators
               WHERE IndicatorName IS NOT NULL;""",
           conn)

filter.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [16]:
# checking for missing values in all columns
indic.isnull().sum()

CountryName      0
CountryCode      0
IndicatorName    0
IndicatorCode    0
Year             0
Value            0
dtype: int64

In [17]:
# how many indicators?
indicCount=pd.read_sql("""SELECT COUNT (DISTINCT IndicatorName)
                FROM Indicators;""",
           conn)

indicCount

,COUNT (DISTINCT IndicatorName)
0,1344


In [18]:
# selecting distinct indicators
distinctIndic=pd.read_sql("""SELECT DISTINCT IndicatorName
                FROM Indicators;""",
           conn)

distinctIndic.head()

,IndicatorName
0,"2005 PPP conversion factor, GDP (LCU per inter..."
1,"2005 PPP conversion factor, private consumptio..."
2,ARI treatment (% of children under 5 taken to ...
3,Access to electricity (% of population)
4,"Access to electricity, rural (% of rural popul..."


# 2- Filtering

In [19]:
# I wanna search for some indicator about GDP
GDPiNDIC=pd.read_sql("""SELECT DISTINCT IndicatorName
               FROM Indicators
               WHERE IndicatorName LIKE 'GDP%';""",
           conn)

GDPiNDIC.head()

,IndicatorName
0,GDP (constant LCU)
1,GDP (current LCU)
2,GDP at market prices (constant 2005 US$)
3,GDP at market prices (current US$)
4,GDP deflator (base year varies by country)


In [23]:
# how about GDP per capita of Brazil in last years ?
BrazilGPD=pd.read_sql(""" SELECT *
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND CountryName = "Brazil"
                AND Year>=2012;""",
           conn)

BrazilGPD.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,2012,11922.513058
1,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,2013,11711.004405
2,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,11384.415010


In [24]:
# let's  compare with China
ChinaGPD=pd.read_sql(""" SELECT *
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND CountryName = "China"
                AND Year>=2012;""",
           conn)

ChinaGPD.head()


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,2012,6264.643878
1,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,2013,6991.853866
2,China,CHN,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,7590.016441


In [25]:
# let's check the 90's in Brazil
Brazil90=pd.read_sql(""" SELECT *
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND CountryName = "Brazil"
                AND Year>=1990;""",
           conn)

Brazil90.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,1990,3071.627953
1,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,1991,3942.403941
2,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,1992,2578.207009
3,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,1993,2774.174127
4,Brazil,BRA,GDP per capita (current US$),NY.GDP.PCAP.CD,1994,3482.529816


In [28]:
# let's check other countries in 2014
Countries2014=pd.read_sql(""" SELECT *
                FROM Indicators
                WHERE Year=2014;""",
           conn)

Countries2014

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,2014,4.822815e+01
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,2014,6.179190e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,2014,6.825914e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,2014,5.332138e+01
4,Arab World,ARB,Agriculture value added per worker (constant 2...,EA.PRD.AGRI.KD,2014,3.353247e+03
...,...,...,...,...,...,...
107384,Zimbabwe,ZWE,Urban population,SP.URB.TOTL,2014,4.955055e+06
107385,Zimbabwe,ZWE,Urban population (% of total),SP.URB.TOTL.IN.ZS,2014,3.250100e+01
107386,Zimbabwe,ZWE,Urban population growth (annual %),SP.URB.GROW,2014,1.837793e+00
107387,Zimbabwe,ZWE,"Use of IMF credit (DOD, current US$)",DT.DOD.DIMF.CD,2014,4.867300e+08


# 3- Aggregating and Summarizing

In [31]:
# let's search for the highest GDP per capita in 2014
HighestGDP=pd.read_sql(""" SELECT CountryName, MAX (Value)
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND Year=2014;""",
           conn)

HighestGDP.head()

,CountryName,MAX (Value)
0,Luxembourg,116664.262488


In [ ]:
#Let's order (and reafirm the highest GDP per capita)
#the default is the ascending order in order by
OrderedHighestGDP=pd.read_sql(""" SELECT *
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND Year=2014
                ORDER BY value DESC 
                Limit 5;""",
           conn)

OrderedHighestGDP

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Luxembourg,LUX,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,116664.262488
1,Norway,NOR,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,97307.421592
2,Qatar,QAT,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,96732.402545
3,"Macao SAR, China",MAC,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,96038.050724
4,Switzerland,CHE,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,85594.326601


In [36]:
# Let's compare the averages
OrderedAVGGDP=pd.read_sql(""" SELECT CountryName, AVG (Value)
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND Year=2014;""",
           conn)

OrderedAVGGDP

,CountryName,AVG (Value)
0,Arab World,14081.842529


In [37]:
#how many measures during this time?
numMeasures=pd.read_sql(""" SELECT COUNT (Value)
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND Year=2014;""",
           conn)


numMeasures

,COUNT (Value)
0,215


In [38]:
# important to know that Angola has less measures during this time
numMeasures=pd.read_sql(""" SELECT COUNT (Value)
                FROM Indicators
                WHERE IndicatorName ='GDP per capita (current US$)'
                AND CountryName='Angola'
                AND Year=2014;""",
           conn)


numMeasures

,COUNT (Value)
0,0


# 4- JOINs

In [39]:
# check the head
country=pd.read_sql("""SELECT *
               FROM Country;""",
           conn)

country.head()

,CountryCode,ShortName,TableName,LongName,Alpha2Code,CurrencyUnit,SpecialNotes,Region,IncomeGroup,Wb2Code,...,GovernmentAccountingConcept,ImfDataDisseminationStandard,LatestPopulationCensus,LatestHouseholdSurvey,SourceOfMostRecentIncomeAndExpenditureData,VitalRegistrationComplete,LatestAgriculturalCensus,LatestIndustrialData,LatestTradeData,LatestWaterWithdrawalData
0,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,Fiscal year end: March 20; reporting period fo...,South Asia,Low income,AF,...,Consolidated central government,General Data Dissemination System (GDDS),1979,"Multiple Indicator Cluster Survey (MICS), 2010/11","Integrated household survey (IHS), 2008",,2013/14,,2013,2000
1,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,,Europe & Central Asia,Upper middle income,AL,...,Budgetary central government,General Data Dissemination System (GDDS),2011,"Demographic and Health Survey (DHS), 2008/09",Living Standards Measurement Study Survey (LSM...,Yes,2012,2011,2013,2006
2,DZA,Algeria,Algeria,People's Democratic Republic of Algeria,DZ,Algerian dinar,,Middle East & North Africa,Upper middle income,DZ,...,Budgetary central government,General Data Dissemination System (GDDS),2008,"Multiple Indicator Cluster Survey (MICS), 2012","Integrated household survey (IHS), 1995",,,2010,2013,2001
3,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,,East Asia & Pacific,Upper middle income,AS,...,,,2010,,,Yes,2007,,,
4,ADO,Andorra,Andorra,Principality of Andorra,AD,Euro,,Europe & Central Asia,High income: nonOECD,AD,...,,,2011. Population data compiled from administra...,,,Yes,,,2006,


In [43]:
# let's make a join to get the information about indicators GDP related measured in 2014 in Brazil
JoinedBrazilGDP=pd.read_sql(""" SELECT Indicators.CountryName, Indicators.Year, Indicators.Value, Country.Region
                FROM Indicators
                INNER JOIN Country ON Indicators.CountryCode=Country.CountryCode
                WHERE Indicators.CountryName='Brazil'
                AND Indicators.Year=2014;""",
           conn)

JoinedBrazilGDP.head()

,CountryName,Year,Value,Region
0,Brazil,2014,67.310600,Latin America & Caribbean
1,Brazil,2014,45.142889,Latin America & Caribbean
2,Brazil,2014,10.995047,Latin America & Caribbean
3,Brazil,2014,34.147841,Latin America & Caribbean
4,Brazil,2014,4.011718,Latin America & Caribbean


# 5- Using CASE WHEN

CASE statements are like "IF this THEN that".  
Here I'm going to use CASE statements for a custom discretization.

In [44]:
# I'm using arbitrary values, this is an exercise with didactic purposes: cuts on 10000 and 80000
didactic=pd.read_sql(""" SELECT *,
                   CASE WHEN Value < 10000 THEN 'Low'
                   WHEN Value > 80000 THEN 'High'
                   ELSE 'Medium' END AS DidacticCategory
                   FROM Indicators
                   WHERE IndicatorName='GDP per capita (current US$)'
                   AND Year=2014;""",
              conn)

didactic.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,DidacticCategory
0,Arab World,ARB,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,7386.428444,Low
1,Caribbean small states,CSS,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,10164.320958,Medium
2,Central Europe and the Baltics,CEB,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,14086.521810,Medium
3,East Asia & Pacific (all income levels),EAS,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,9475.349629,Low
4,East Asia & Pacific (developing only),EAP,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,6240.403751,Low


# 6 - Window Function
With a window function, we can make calculations across rows "in a window" and return a value for each row.
This window can be, for example, grouped sets based on another column or even an ordered set.

In [46]:
# adding the mean of last 3 years using window function
# these are the years I found answers for them (not exactly the last 3 years)
mean3Y=pd.read_sql(""" SELECT *,
                ROUND(AVG(Value) OVER (PARTITION BY CountryCode),0) AS AVG_3Y
                FROM Indicators
                WHERE IndicatorName='GDP per capita (current US$)'
                AND Year IN (2012,2013,2014);""",
           conn)

mean3Y.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,AVG_3Y
0,Andorra,ADO,GDP per capita (current US$),NY.GDP.PCAP.CD,2012,39666.369215,41236.0
1,Andorra,ADO,GDP per capita (current US$),NY.GDP.PCAP.CD,2013,42806.522553,41236.0
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,2012,690.842629,664.0
3,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,2013,666.795051,664.0
4,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,633.569247,664.0


# 7 - Common Table Expression (CTE)
A CTE will save results of a query temporary. It can help simplifying some queries and, for example, can also help filtering based on ranking results. 

In [50]:
# the first place from each year
# creates a Common Table Expression (CTE) named place_1
Rank1 = pd.read_sql("""
    WITH place_1 AS (
        SELECT *,
               RANK() OVER (PARTITION BY Year ORDER BY Value DESC) AS Rank
        FROM Indicators
        WHERE IndicatorName = 'GDP per capita (current US$)'
          AND Year IN (2012, 2013, 2014)
    )
    SELECT *
    FROM place_1
    WHERE Rank = 1;
    """, conn)

Rank1

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,Rank
0,Liechtenstein,LIE,GDP per capita (current US$),NY.GDP.PCAP.CD,2012,149160.758132,1
1,Luxembourg,LUX,GDP per capita (current US$),NY.GDP.PCAP.CD,2013,113726.638979,1
2,Luxembourg,LUX,GDP per capita (current US$),NY.GDP.PCAP.CD,2014,116664.262488,1
